# Análise de Times do League of Legends

### Importação das principais bibliotecas

In [85]:
from mwrogue.esports_client import EsportsClient
import pandas as pd

### Fazer o request na API do lol fandom

In [87]:
site = EsportsClient("lol")

response = site.cargo_client.query(
    tables = "Tournaments=T, Leagues=L, MatchSchedule=M",
    fields="L.League_Short=League, T.Name=Tournament, M.DateTime_UTC=Date, M.Team1Final=Team1, M.Team2Final=Team2, M.Team1Score, M.Team2Score",
    where = 'L.League_Short = "CBLOL"',
    join_on = "T.League = L.League, T.OverviewPage=M.OverviewPage"
)

### Criação do dataframe e pre processamento

In [88]:
MatchData = pd.DataFrame(response)

In [92]:
MatchData = MatchData.replace("",float("NaN")).dropna(axis='index', how='any')
MatchData = MatchData.drop(['League', 'Tournament'], axis=1)
MatchData = MatchData.astype({'Team1Score':'int32','Team2Score':'int32'})

#### Arrumando a data e fazendo

In [94]:
MatchData['Date'] = pd.to_datetime(MatchData['Date'])
MatchData['Date'] = MatchData['Date'].dt.date
MatchData = MatchData.drop("Date__precision", axis=1)
MatchData.sort_values(by="Date",inplace=True)

In [95]:
print(MatchData)

            Date              Team1            Team2 Team1Score Team2Score
0     2014-04-26     KaBuM! Esports      Team United          1          1
1     2014-04-26         Keyd Stars         Team awp          2          0
2     2014-04-27        paiN Gaming      Team United          0          2
3     2014-04-27         Keyd Stars   KaBuM! Esports          2          0
4     2014-04-27  CNB e-Sports Club         Team awp          1          1
...          ...                ...              ...        ...        ...
1256  2023-08-19               INTZ  Vivo Keyd Stars          3          1
1257  2023-08-20        paiN Gaming             LOUD          0          3
1258  2023-08-26         RED Canids             INTZ          3          1
1259  2023-08-27        paiN Gaming       RED Canids          3          2
1260  2023-09-09               LOUD      paiN Gaming          3          1

[1263 rows x 5 columns]


#### Arrundo o sistema de Partidas

In [102]:
MatchData['Team1'] = MatchData['Team1'].replace('Keyd Stars', 'Vivo Keyd Stars')
MatchData['Team1'] = MatchData['Team1'].replace('Vivo Keyd', 'Vivo Keyd Stars')
MatchData['Team1'] = MatchData['Team1'].replace('Keyd Warriors', 'Vivo Keyd Stars')

MatchData['Team2'] = MatchData['Team2'].replace('Keyd Stars', 'Vivo Keyd Stars')
MatchData['Team2'] = MatchData['Team2'].replace('Vivo Keyd', 'Vivo Keyd Stars')
MatchData['Team2'] = MatchData['Team2'].replace('Keyd Warriors', 'Vivo Keyd Stars')

MatchData['Team1'] = MatchData['Team1'].replace('KaBuM! Orange', 'KaBuM! Esports')
MatchData['Team2'] = MatchData['Team2'].replace('KaBuM! Orange', 'KaBuM! Esports')

MatchData['Team1'] = MatchData['Team1'].replace('Vorax Liberty', 'Liberty')
MatchData['Team2'] = MatchData['Team2'].replace('Vorax Liberty', 'Liberty')

MatchData['Team1'] = MatchData['Team1'].replace('FURIA Uppercut', 'FURIA')
MatchData['Team2'] = MatchData['Team2'].replace('FURIA Uppercut', 'FURIA')

MatchData['Team1'] = MatchData['Team1'].replace('Flamengo Los Grandes', 'Los Grandes')
MatchData['Team2'] = MatchData['Team2'].replace('Flamengo Los Grandes', 'Los Grandes')


In [101]:
team_performance = {}
for index, row in MatchData.iterrows():
    team1, team2 = row['Team1'], row['Team2']
    score1, score2 = row['Team1Score'], row['Team2Score']

    # Initialize dictionary for new teams
    if team1 not in team_performance:
        team_performance[team1] = {'wins': 0, 'losses': 0, 'draws': 0}
    if team2 not in team_performance:
        team_performance[team2] = {'wins': 0, 'losses': 0, 'draws': 0}

    # Update records
    if score1 > score2:
        team_performance[team1]['wins'] += 1
        team_performance[team2]['losses'] += 1
    elif score1 < score2:
        team_performance[team1]['losses'] += 1
        team_performance[team2]['wins'] += 1
    else:
        team_performance[team1]['draws'] += 1
        team_performance[team2]['draws'] += 1

# Convert to DataFrame for visualization
team_performance_df = pd.DataFrame.from_dict(team_performance, orient='index')
team_performance_df = team_performance_df.sort_values(by=['wins'], ascending=False)
print(team_performance_df)

                                 wins  losses  draws
paiN Gaming                       147      93     27
INTZ                              132     132     12
KaBuM! Esports                    128     136     11
RED Canids                        106      63      7
Flamengo Esports                  100      56      0
Vivo Keyd Stars                    91      82     17
LOUD                               77      43      0
FURIA                              75      84      0
CNB e-Sports Club                  50      56     21
Liberty                            38      55      0
Redemption eSports Porto Alegre    25      39      0
Los Grandes                        25      32      0
Rensga Esports                     24      51      0
Prodigy Esports                    22      21      0
Uppercut esports                   21      22      0
Fluxo                              21      17      0
Netshoes Miners                    15      22      0
Team oNe eSports                   15      25 